In [ ]:
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
from matplotlib import cm

In [ ]:
def my_logpdf(x, u, covar):
    """This function calculates the value of a multivariate Gaussian pdf at a given point x, with mean u and covariance covar

    Args:
        x : The point at which the pdf is evaluated. Nx1 column vector.
        u : The mean vector of the Gaussian distribution. Nx1 column vector.
        covar : The covariance matrix of the Gaussian distribution. NxN matrix.

    Returns:
        The value of the Gaussian pdf at x.
    """
    k = len(x)  # dimension
    a = np.transpose(x - u)
    b = np.linalg.inv(covar)
    c = x - u
    d = np.matmul(a, b)
    e = np.matmul(d, c)
    numer = np.exp(-0.5 * e)
    f = (2 * np.pi)**k
    g = np.linalg.det(covar)
    denom = np.sqrt(f * g)
    pdf = numer / denom
    return pdf

In [ ]:
# Original state cost function
def state_cost(state,goal_points,obs_points):
    v = np.array([0.02, 0.02], dtype=np.float32)
    covar = np.diag(v)
    gauss_sum = 0

    for i in range(np.size(obs_points,axis=1)):
        gauss_sum += 20*my_logpdf(state[:2],obs_points[:2,i],covar)

    cost = 30*((state[0]-goal_points[0])**2 + (state[1]-goal_points[1])**2) + gauss_sum + 10*(np.exp(-0.5*((state[0]-(-1.5))/0.02)**2)/(0.02*np.sqrt(2*np.pi))
                + np.exp(-0.5*((state[0]-1.5)/0.02)**2)/(0.02*np.sqrt(2*np.pi)) + np.exp(-0.5*((state[1]-1.0)/0.02)**2)/(0.02*np.sqrt(2*np.pi))
                + np.exp(-0.5*((state[1]-(-1.0))/0.02)**2)/(0.02*np.sqrt(2*np.pi)))
    return(cost)

base_obstacle = lambda x,y, obstacle: 20*my_logpdf(np.array([x,y]),obstacle,np.diag(np.array([0.02, 0.02], dtype=np.float32))) # Obstacle cost function

In [ ]:
#Utility functions
def generate_space(lim_x, lim_y, n_points_x, n_points_y):
    x = np.linspace(lim_x[0],lim_x[1],n_points_x)
    y = np.linspace(lim_y[0],lim_y[1],n_points_y)
    X, Y = np.meshgrid(x,y)
    return x, y, X, Y

def eval_2D_func(x_axis, y_axis, fun, args={}):
    n_points_x = len(x_axis)
    n_points_y = len(y_axis)
    Z = np.zeros((n_points_x,n_points_y))
    for i in range(n_points_x):
        for j in range(n_points_y):
            Z[i,j] += fun(x_axis[i],y_axis[j], **args)
    return Z

# Dropwave Tuner

In [ ]:
base_goal = lambda X,Y, goal: 30*((X-goal[0])**2 + (Y-goal[1])**2)
dropwave = lambda X,Y,f,s,p,o,k,A, obstacle: A*(o + np.cos(f*np.sqrt(((X-obstacle[0])/k)**2+((Y-obstacle[1])/k)**2)-f*np.sqrt((obstacle[0]/k)**2+(obstacle[1]/k)**2))/((s)*((X-obstacle[0])**2+(Y-obstacle[1])**2)+p))
dropwave_attenuation = lambda X,Y,f,s,p,o,k,A,goal,obstacle,alpha,beta: dropwave(X,Y,f,s,p,o,k,A, obstacle) * np.exp(alpha*np.sqrt((X-goal[0])**2+(Y-goal[1])**2)) * np.exp(-beta*np.sqrt((X-obstacle[0])**2+(Y-obstacle[1])**2))

In [ ]:
f = 1
s = 1
p = 1
o = 1
k = 1
A = 1
zita = 1
alpha = 1
beta = 1
goal = np.array(np.mat('1.0; 0.0; 0'))
obstacles = np.array(np.mat('0.0 0.0 0.0; 0.0 -0.3 0.3; 0 0 0 '))

In [ ]:
%matplotlib tk
n_points_x = 300
n_points_y = 200

x_axis, y_axis, X, Y = generate_space([-1.5, 1.5], [-1.0, 1.0], n_points_x, n_points_y)
base_obstacle_map = sum([eval_2D_func(x_axis, y_axis, base_obstacle, {'obstacle': obstacle}) for obstacle in obstacles[:2,:].T])

In [ ]:
state_cost_map = eval_2D_func(x_axis, y_axis, lambda x,y,goal_points,obs_points: state_cost(np.array([x,y]),goal_points,obs_points), {'goal_points': goal, 'obs_points': obstacles})

In [ ]:
cost = sum([dropwave_attenuation(X,Y,f,s,p,o,k,A,goal, obstacle, alpha , beta) for obstacle in obstacles[:2,:].T]) + base_goal(X,Y,goal) + base_obstacle_map.T 
cost_gradient = np.gradient(cost)
cost_gradient_module = np.sqrt(cost_gradient[0]**2 + cost_gradient[1]**2)

fig = plt.figure(figsize=(20,20))
ax = fig.add_subplot(2,1,1,projection='3d')

#p: Inversamente proprozionale al picco
#s: Inversamente proprozionale al cratere
#f: legato alla frequenza
#o: Un offset
surf = ax.plot_surface(X,Y,cost, cmap=cm.coolwarm)
# fix axis to robotarium size
# ax.set_xlim(-1.5,1.5)
# ax.set_ylim(-1,1)
# ax.set_zlim(-10,100)

ax2 = fig.add_subplot(2,1,2)
heathmap = ax2.pcolormesh(X,Y,cost_gradient_module, vmin=0, vmax=2, cmap='RdGy')
fig.colorbar(heathmap, ax=ax2)

In [ ]:
import ipywidgets as widgets
from IPython.display import display

# Define the sliders
f_slider = widgets.FloatSlider(min=0.00001, max=500.0, step=0.00001, value=100, description='f:')
s_slider = widgets.FloatSlider(min=0.0001, max=100.0, step=0.001, value=100, description='s:')
p_slider = widgets.FloatSlider(min=0.0001, max=250.0, step=0.001, value=30.0, description='p:')
o_slider = widgets.FloatSlider(min=0.0, max=10.0, step=0.001, value=0.0, description='o:')
k_slider = widgets.FloatSlider(min=0.0, max=20.0, step=0.001, value=7.41, description='k:')
A_slider = widgets.FloatSlider(min=0.0, max=100.0, step=0.001, value=37.16, description='A:')
alpha_slider = widgets.FloatSlider(min=0.0, max=20.0, step=0.00001, value=1.0, description='alpha:')
beta_slider = widgets.FloatSlider(min=0, max=30.0, step=0.00001, value=1.0, description='beta:')

# Display the sliders
display(f_slider, s_slider, p_slider, o_slider, k_slider, A_slider, alpha_slider, beta_slider)

# Update the function to use the slider values
def update_plot():
    f = f_slider.value
    s = s_slider.value
    p = p_slider.value
    o = o_slider.value
    k = k_slider.value
    A = A_slider.value
    alpha = alpha_slider.value
    beta = beta_slider.value
    
    cost = sum([dropwave_attenuation(X,Y,f,s,p,o,k,A,goal, obstacle, alpha , beta) for obstacle in obstacles[:2,:].T]) + base_obstacle_map.T + base_goal(X,Y,goal)
    cost_gradient = np.gradient(cost)
    cost_gradient_module = np.sqrt(cost_gradient[0]**2 + cost_gradient[1]**2)
    
    
    ax.clear()
    surf = ax.plot_surface(X,Y,cost, cmap=cm.coolwarm)
    ax.set_xlim(-1.5,1.5)
    ax.set_ylim(-1,1)
    #ax.set_zlim(-5,600)
    
    ax2.clear()
    heathmap = ax2.pcolormesh(X,Y,cost_gradient_module, vmin=0, vmax=2, cmap='RdGy')


# Call the update function when a slider value changes
f_slider.observe(lambda change: update_plot(), names='value')
s_slider.observe(lambda change: update_plot(), names='value')
p_slider.observe(lambda change: update_plot(), names='value')
o_slider.observe(lambda change: update_plot(), names='value')
k_slider.observe(lambda change: update_plot(), names='value')
A_slider.observe(lambda change: update_plot(), names='value')
alpha_slider.observe(lambda change: update_plot(), names='value')
beta_slider.observe(lambda change: update_plot(), names='value')

# Initial plot
update_plot()

In [ ]:
print("'f': %f," % f_slider.value)
print("'s': %f," % s_slider.value)
print("'p': %f," % p_slider.value)
print("'o': %f," % o_slider.value)
print("'k': %f," % k_slider.value)
print("'A': %f," % A_slider.value)
print("'alpha': %f," % alpha_slider.value)
print("'beta': %f," % beta_slider.value)

# Single Wave Tuner

In [ ]:
exp_distance = lambda X,Y,Lambda: np.exp(Lambda*np.sqrt(X**2+Y**2))
euclidean_distance = lambda X,Y: np.sqrt(X**2+Y**2)
problem06=lambda X,Y: - (X + np.sin(X))*np.exp(-X**2)
single_wave = lambda X,Y,goal, A, k,omega,Lambda, alpha: A*(exp_distance(X,Y,Lambda)/k**2+np.sin(omega*exp_distance(X,Y,Lambda)/k**2))*np.exp(-(exp_distance(X,Y,Lambda)/k**2)**2) * np.exp(alpha*np.sqrt((X-goal[0])**2+(Y-goal[1])**2))
sqare_waves = lambda X,Y, A, k, omega: A*np.cos(omega*np.abs(X/k)+np.abs(Y/k))
circular_waves = lambda X,Y, A, k,omega: A*np.cos(omega*np.sqrt((X/k)**2 + (Y/k)**2))

goal = np.array(np.mat('1.0; 0.0; 0'))
obstacles = np.array(np.mat('0.0; 0.0; 0.0'))#np.array(np.mat('0.0 0.0 0.0; 0.0 -0.3 0.3; 0 0 0 '))

In [ ]:
%matplotlib tk
n_points_x = 300
n_points_y = 200

x_axis, y_axis, X, Y = generate_space([-1.5, 1.5], [-1, 1], n_points_x, n_points_y)
base_obstacle_map = sum([eval_2D_func(x_axis, y_axis, base_obstacle, {'obstacle': obstacle}) for obstacle in obstacles[:2,:].T])

In [ ]:
A=1
k=1
omega=1
k2=1
B=1
slope=1
Lambda=1
alpha=1

In [ ]:
cost = single_wave(X,Y,goal, A, k,omega,Lambda, alpha)
cost_gradient = np.gradient(cost)
cost_gradient_module = np.sqrt(cost_gradient[0]**2 + cost_gradient[1]**2)

fig = plt.figure(figsize=(12,12))
ax = fig.add_subplot(2,1,1,projection='3d')

surf = ax.plot_surface(X,Y,cost, cmap=cm.coolwarm)
# fix axis to robotarium size
#ax.set_xlim(-1.5,1.5)
#ax.set_ylim(-1,1)
#ax.set_zlim(-10,100)

ax2 = fig.add_subplot(2,1,2)
heathmap = ax2.pcolormesh(X,Y,cost_gradient_module, vmin=0, vmax=2, cmap='RdGy')
fig.colorbar(heathmap, ax=ax2)

In [ ]:
import ipywidgets as widgets
from IPython.display import display

# Define the sliders
A_slider = widgets.FloatSlider(min=-100, max=100.0, step=0.00001, value=1, description='A:')
k_slider = widgets.FloatSlider(min=-5, max=5000, step=0.00001, value=1, description='k:')
omega_slider = widgets.FloatSlider(min=0, max=10, step=0.00001, value=1, description='omega:')
B_slider = widgets.FloatSlider(min=-10, max=10.0, step=0.00001, value=1, description='B:')
slope_slider = widgets.FloatSlider(min=-10, max=10.0, step=0.00001, value=1, description='slope:')
lambda_slider = widgets.FloatSlider(min=-10, max=30.0, step=0.00001, value=1, description='lambda:')
alpha_slider = widgets.FloatSlider(min=-10, max=30.0, step=0.00001, value=1, description='alpha:')
# Display the sliders
display(A_slider, k_slider,omega_slider, B_slider, slope_slider, lambda_slider, alpha_slider)

# Update the function to use the slider values
def update_plot():
    A = A_slider.value
    k = k_slider.value
    omega = omega_slider.value
    B=B_slider.value
    slope=slope_slider.value
    Lambda=lambda_slider.value
    alpha=alpha_slider.value
    
    cost = single_wave(X,Y,goal, A, k,omega,Lambda, alpha)
    cost_gradient = np.gradient(cost)
    cost_gradient_module = np.sqrt(cost_gradient[0]**2 + cost_gradient[1]**2)
    
    
    ax.clear()
    surf = ax.plot_surface(X,Y,cost, cmap=cm.coolwarm)
    #ax.set_xlim(-1.5,1.5)
    #ax.set_ylim(-1,1)
    #ax.set_zlim(-5,100)
    
    ax2.clear()
    heathmap = ax2.pcolormesh(X,Y,cost_gradient_module, vmin=0, vmax=2, cmap='RdGy')


# Call the update function when a slider value changes
A_slider.observe(lambda change: update_plot(), names='value')
k_slider.observe(lambda change: update_plot(), names='value')
omega_slider.observe(lambda change: update_plot(), names='value')
B_slider.observe(lambda change: update_plot(), names='value')
slope_slider.observe(lambda change: update_plot(), names='value')
lambda_slider.observe(lambda change: update_plot(), names='value')
alpha_slider.observe(lambda change: update_plot(), names='value')
# Initial plot
update_plot()

In [ ]:
print("'A': %f," % A_slider.value)
print("'k': %f," % k_slider.value)
print("'omega': %f," % omega_slider.value)
print("'L': %f," % lambda_slider.value)
print("'alpha': %f," % alpha_slider.value)

# Circular gaussian wave tuner

In [ ]:
import numpy as np
rho = 0.7
var_x = var_y = np.sqrt(0.02)
cov = np.array([[var_x**2, rho*var_x*var_y], [rho*var_x*var_y, var_y**2]])

base_obstacle = lambda x,y, obstacle: 20*my_logpdf(np.array([x,y]),obstacle,cov) # Obstacle cost function

base_goal = lambda X,Y, goal: 30*((X-goal[0])**2 + (Y-goal[1])**2)
exp_distance = lambda X,Y,Lambda: np.exp(Lambda*np.sqrt(X**2+Y**2))
euclidean_distance = lambda X,Y,O=np.array([0,0]): np.sqrt((X-O[0])**2+(Y-O[1])**2)
vector_module = lambda X,Y: np.sqrt(X**2+Y**2)
gaussian = lambda X, mu, sigma: np.exp(-0.5*((X-mu)/sigma)**2)/(sigma*np.sqrt(2*np.pi))
theta = lambda X,Y,G,O: np.arccos(((X-G[0])*(O[0]-G[0]) + (Y-G[1])*(O[1]-G[1]))/(vector_module(X-G[0],Y-G[1])+0.00001)/(vector_module(O[0]-G[0],O[1]-G[1])+0.00001))
decay = lambda X,Y,G,O,l: np.exp(-l*(theta(X,Y,G,O)/np.pi)**2)
circular_barrier = lambda X,Y,center,r,sigma,A,obstacle,l: A*gaussian(euclidean_distance(X,Y,center),r+euclidean_distance(obstacle[0],obstacle[1],center),sigma)*decay(X,Y,center,obstacle,l)

goal = np.array(np.mat('1.0; 0.0; 0'))
obstacles = np.array(np.mat('0.0 0.0 0.0; 0.0 -0.3 0.3; 0 0 0'))


In [ ]:
%matplotlib tk
n_points_x = 300
n_points_y = 200

x_axis, y_axis, X, Y = generate_space([-1.5, 1.5], [-1, 1], n_points_x, n_points_y)
base_obstacle_map = sum([eval_2D_func(x_axis, y_axis, base_obstacle, {'obstacle': obstacle}) for obstacle in obstacles[:2,:].T])

In [ ]:
r = 1
sigma = 1
A = 1
l=-0.5
center = np.array(np.mat('1.0; 0.0; 0'))

In [ ]:
#cost = sum([circular_barrier(X,Y,center,r,sigma,A,obstacle,l) for obstacle in obstacles[:2,:].T]) + base_obstacle_map.T + base_goal(X,Y,goal)
cost = base_obstacle_map.T +base_goal(X,Y,goal)

cost_gradient = np.gradient(cost)
cost_gradient_module = np.sqrt(cost_gradient[0]**2 + cost_gradient[1]**2)

fig = plt.figure(figsize=(12,12))
ax = fig.add_subplot(2,1,1,projection='3d')

surf = ax.plot_surface(X,Y,cost, cmap=cm.coolwarm)
# fix axis to robotarium size
#ax.set_xlim(-1.5,1.5)
#ax.set_ylim(-1,1)
#ax.set_zlim(-10,100)

ax2 = fig.add_subplot(2,1,2)
heathmap = ax2.pcolormesh(X,Y,cost_gradient_module, vmin=0, vmax=2, cmap='RdGy')
fig.colorbar(heathmap, ax=ax2)

In [ ]:
import ipywidgets as widgets
from IPython.display import display

# Define the sliders
r_slider = widgets.FloatSlider(min=0, max=2.0, step=0.00001, value=1, description='r:')
sigma_slider = widgets.FloatSlider(min=0, max=0.20, step=0.00001, value=0.1, description='sigma:')
A_slider = widgets.FloatSlider(min=0, max=50, step=0.00001, value=1, description='A:')
l_slider = widgets.FloatSlider(min=0, max=300, step=0.00001, value=-0.5, description='l:')
# Display the sliders
display(r_slider, sigma_slider, A_slider, l_slider)

# Update the function to use the slider values
def update_plot():
    r = r_slider.value
    sigma = sigma_slider.value
    A = A_slider.value
    l = l_slider.value
    cost = sum([circular_barrier(X,Y,center,r,sigma,A,obstacle,l) for obstacle in obstacles[:2,:].T]) + base_obstacle_map.T + base_goal(X,Y,goal)
    cost_gradient = np.gradient(cost)
    cost_gradient_module = np.sqrt(cost_gradient[0]**2 + cost_gradient[1]**2)
    
    
    ax.clear()
    surf = ax.plot_surface(X,Y,cost, cmap=cm.coolwarm)
    #ax.set_xlim(-1.5,1.5)
    #ax.set_ylim(-1,1)
    #ax.set_zlim(-5,100)
    
    ax2.clear()
    heathmap = ax2.pcolormesh(X,Y,cost_gradient_module, vmin=0, vmax=2, cmap='RdGy')


# Call the update function when a slider value changes
r_slider.observe(lambda change: update_plot(), names='value')
sigma_slider.observe(lambda change: update_plot(), names='value')
A_slider.observe(lambda change: update_plot(), names='value')
l_slider.observe(lambda change: update_plot(), names='value')
# Initial plot
update_plot()

In [ ]:
print("'r': %f," % r_slider.value)
print("'sigma': %f," % sigma_slider.value)
print("'A': %f," % A_slider.value)
print("'l': %f," % l_slider.value)

# Circular gaussian wave tuner

In [ ]:
import numpy as np
from scipy.stats import multivariate_normal

def bivariate_gaussian(x, y, mean, var_x, var_y, rho):
    covariance_matrix = np.array([[var_x, rho*np.sqrt(var_x)*np.sqrt(var_y)], [rho*np.sqrt(var_x)*np.sqrt(var_y), var_y]])
    
    pos = np.dstack((x, y))
    rv = multivariate_normal(mean, covariance_matrix,allow_singular=True)
    # Calculate the probability density function (PDF) values
    z = rv.pdf(pos)

    return z


base_goal = lambda X,Y, goal: 30*( (X-goal[0])**2 + (Y-goal[1])**2 -1/((X-goal[0])**2 + (Y-goal[1])**2 +0.1))
exp_distance = lambda X,Y,Lambda: np.exp(Lambda*np.sqrt(X**2+Y**2))
euclidean_distance = lambda X,Y,O=np.array([0,0]): np.sqrt((X-O[0])**2+(Y-O[1])**2)
vector_module = lambda X,Y: np.sqrt(X**2+Y**2)
gaussian = lambda X, mu, sigma: np.exp(-0.5*((X-mu)/sigma)**2)/(sigma*np.sqrt(2*np.pi))

theta = lambda X,Y,G,O: np.arccos(((X-G[0])*(O[0]-G[0]) + (Y-G[1])*(O[1]-G[1]))/(vector_module(X-G[0],Y-G[1])+0.00001)/(vector_module(O[0]-G[0],O[1]-G[1])+0.00001))

decay = lambda X,Y,G,O,l: np.exp(-l*(theta(X,Y,G,O)/np.pi)**2)


goal = np.array(np.mat('1.0; 0.0; 0'))
obstacles = np.array(np.mat('0.0 0.0 0.0; 0.0 -0.3 0.3; 0 0 0'))


In [ ]:
%matplotlib tk
n_points_x = 300
n_points_y = 200

x_axis, y_axis, X, Y = generate_space([-1.5, 1.5], [-1, 1], n_points_x, n_points_y)
base_obstacle_map = sum([eval_2D_func(x_axis, y_axis, base_obstacle, {'obstacle': obstacle}) for obstacle in obstacles[:2,:].T])

In [ ]:
rho = 0
var_x = var_y = 0.02
A = 1
covariance_matrix = np.array([[var_x, rho*np.sqrt(var_x)*np.sqrt(var_y)], [rho*np.sqrt(var_x)*np.sqrt(var_y), var_y]])
covariance_matrix

In [ ]:
#cost = sum([circular_barrier(X,Y,center,r,sigma,A,obstacle,l) for obstacle in obstacles[:2,:].T]) + base_obstacle_map.T + base_goal(X,Y,goal)
cost = sum([A*bivariate_gaussian(X, Y,obstacle, var_x, var_y,rho)+5*bivariate_gaussian(X, Y,obstacle, 0.005, 0.005,0) for obstacle in obstacles[:2,:].T]) + base_goal(X,Y,goal)
# 
cost_gradient = np.gradient(cost)
cost_gradient_module = np.sqrt(cost_gradient[0]**2 + cost_gradient[1]**2)

fig = plt.figure(figsize=(12,12))
ax = fig.add_subplot(2,1,1,projection='3d')

surf = ax.plot_surface(X,Y,cost, cmap=cm.coolwarm)
# fix axis to robotarium size
# ax.set_xlim(-1.5,1.5)
# ax.set_ylim(-1,1)
# ax.set_zlim(-10,100)

ax2 = fig.add_subplot(2,1,2)
heathmap = ax2.pcolormesh(X,Y,cost_gradient_module, vmin=0, vmax=2, cmap='RdGy')
fig.colorbar(heathmap, ax=ax2)

In [ ]:
import ipywidgets as widgets
from IPython.display import display

# Define the sliders
rho_slider = widgets.FloatSlider(min=-1, max=1, step=0.00001, value=0.0, description='rho:')
var_x_slider = widgets.FloatSlider(min=0, max=1, step=0.00001, value=0.1, description='var_x:')
var_y_slider = widgets.FloatSlider(min=0, max=1, step=0.00001, value=0.1, description='var_y:')
A_slider = widgets.FloatSlider(min=0, max=100, step=0.00001, value=20, description='A:')
# Display the sliders
display(rho_slider, var_x_slider, var_y_slider, A_slider)

# Update the function to use the slider values
def update_plot():
    
    rho = rho_slider.value
    var_x = var_x_slider.value
    var_y = var_y_slider.value
    A = A_slider.value
    cost = sum([A*bivariate_gaussian(X, Y,obstacle, var_x, var_y,rho)+5*bivariate_gaussian(X, Y,obstacle, 0.008, 0.008,0) for obstacle in obstacles[:2,:].T]) + base_goal(X,Y,goal)

    cost_gradient = np.gradient(cost)
    cost_gradient_module = np.sqrt(cost_gradient[0]**2 + cost_gradient[1]**2)
    
    
    ax.clear()
    surf = ax.plot_surface(X,Y,cost, cmap=cm.coolwarm)
    #ax.set_xlim(-1.5,1.5)
    #ax.set_ylim(-1,1)
    #ax.set_zlim(-5,100)
    
    ax2.clear()
    heathmap = ax2.pcolormesh(X,Y,cost_gradient_module, vmin=0, vmax=2, cmap='RdGy')


# Call the update function when a slider value changes
rho_slider.observe(lambda change: update_plot(), names='value')
var_x_slider.observe(lambda change: update_plot(), names='value')
var_y_slider.observe(lambda change: update_plot(), names='value')
A_slider.observe(lambda change: update_plot(), names='value')
# Initial plot
update_plot()

In [ ]:
print("'rho': %f," % rho_slider.value)
print("'var_x': %f," % var_x_slider.value)
print("'var_y': %f," % var_y_slider.value)
print("'A': %f" % A_slider.value)
